# Analyzing Trace Statistics

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from MemoryAutoScaling import analysis, plotting, specs, utils

pd.set_option('display.max_columns', None)

### Function Definitions

In [ ]:
def plot_utilization_cdf(stat_df, resource_name):
    """Plots a CDF of utilization for `resource_name` from `stat_df`.
    
    Parameters
    ----------
    stat_df: pd.DataFrame
        A pandas DataFrame containing the utilization statistics.
    resource_name: str
        A string representing the name of the resource to which the utilization rates apply.
    
    Returns
    -------
    None
    
    """
    utilization_vals = utils.impute_for_time_series(stat_df['utilization'].values, 0.0)
    plt.figure(figsize=(20, 10))
    x, y = plotting.get_cdf_values(utilization_vals)
    plt.plot(x, y)
    plt.title("CDF of {} Utilization Rates Across Traces".format(resource_name))
    plt.xlabel("Utilization Rate")
    plt.show()

In [ ]:
def plot_all_correlations_for_lag(stat_df, stats_col, lag):
    """Plots all correlations for `stats_col` in `stat_df` at `lag`.
    
    Parameters
    ----------
    stat_df: pd.DataFrame
        A pandas DataFrame containing the correlation statistics for `stats_col`.
    stats_col: str
        A string representing the column for which the correlations are computed. Each
        correlation is another variable versus `stats_col`.
    lag: int
        An integer representing the lag used in the correlation calculations. For each
        correlation plotted, the correlation is between `stats_col` and another variable
        lagged by `lag` time steps.
    
    Returns
    -------
    None
    
    """
    plt.figure(figsize=(20, 10))
    for col_name in specs.get_trace_columns():
        x, y = plotting.get_cdf_values(stat_df["corr_{0}_lag_{1}".format(col_name, lag)].abs().values)
        plt.plot(x, y,label=col_name)
    plt.title("{0} Usage Correlations - Lag {1}".format(stats_col, lag))
    plt.legend()
    plt.show()

In [ ]:
def plot_all_correlations_across_all_lags(stat_df, stats_col):
    """Plots all correlations for `stats_col` in `stat_df` across all lags.
    
    Parameters
    ----------
    stat_df: pd.DataFrame
        A pandas DataFrame containing the correlation statistics for `stats_col`.
    stats_col: str
        A string representing the column for which the correlations are computed. Each
        correlation is another variable versus `stats_col`.
    
    Returns
    -------
    None
    
    """
    for lag in [1, 2, 3]:
        plot_all_correlations_for_lag(stat_df, stats_col, lag)

In [ ]:
def print_stationarity_results(stat_df):
    """Prints the results of the stationarity tests in `stat_df`.
    
    Parameters
    ----------
    stat_df: pd.DataFrame
        A pandas DataFrame containing the statistics for traces, including
        tests for stationarity in columns `adf_p_val`, `adf_p_val_diff` and
        `adf_p_val_diff_2`.
    
    Returns
    -------
    None
    
    """
    print("Stationarity Results")
    stationary_dict = analysis.stationary_results_from_stats_df(stat_df)
    print("Number of Stationary Traces: {}%".format(stationary_dict['diff_0']))
    print("Number of Stationary Traces after 1 Level Differencing: {}%".format(stationary_dict['diff_1']))
    print("Number of Stationary Traces after 2 Level Differencing: {}%".format(stationary_dict['diff_2']))
    print("Other: {}%".format(stationary_dict['other']))

In [ ]:
def print_correlation_results(stat_df, target_col, corr_cols):
    """Prints a summary of correlation results for `target_col` vs other time series.
    
    Parameters
    ----------
    stat_df: pd.DataFrame
        A pandas DataFrame containing the statistics for traces.
    target_col: str
        A string representing the name of the target column for which the correlations are generated.
    corr_cols: list
        A list of columns for which the correlation results are retrieved.
    
    Returns
    -------
    None
    
    """
    print("Correlation Results")
    for col_name in corr_cols:
        corr_count = len(stat_df[stat_df["corr_{}_ts".format(col_name)].abs() >= 0.7])
        print("Number of traces with correlation of {0} and {1} >= 0.7 or <= -0.7: {2}".format(
            target_col, col_name, corr_count))

In [ ]:
def print_statistical_analysis_summary(stat_df, target_col, corr_cols):
    """Prints a summary of the results of `stat_df` for `target_col`.
    
    Parameters
    ----------
    stat_df: pd.DataFrame
        A pandas DataFrame containing the statistics for traces.
    target_col: str
        A string representing the name of the target column for which the statistics are generated.
    corr_cols: list
        A list of columns for which the correlation results are retrieved.
    
    Returns
    -------
    None
    
    """
    print("Total Trace Count: {}".format(len(stat_df)))
    print()
    print_stationarity_results(stat_df)
    print()
    print_correlation_results(stat_df, target_col, corr_cols)

In [ ]:
def plot_lagged_cdfs(data_df, col_name, col_title, stats_col):
    """Plots the lagged CDFs for `col_name`.
    
    Parameters
    ----------
    data_df: pd.DataFrame
        The pandas DataFrame containing the distributions for the CDFs
    col_name: str
        A string representing the column for which the CDFs are generated.
    col_title: str
        A string representing the column title for the CDFs.
    stats_col: str
        A string representing the name of the column for which all statistics
        are generated.
    
    Returns
    -------
    None
    
    """
    fig, axes = plt.subplots(2, 2, figsize=(20, 8))
    lags = [0, 1, 2, 3]
    col_names = ["corr_{}".format(col_name)] + ["corr_{0}_lag_{1}".format(col_name, lag) for lag in lags[1:]]
    colors = ["blue", "black", "green", "red"]
    for idx in range(4):
        row = idx // 2
        col = idx % 2
        data_vals = data_df[col_names[idx]].abs().values
        data_vals = data_vals[~np.isnan(data_vals)]
        plotting.plot_cumulative_distribution_function(data_vals, axes[row, col],
                                                       "{0} Lag {1}".format(col_title, lags[idx]), colors[idx],
                                                       "CDF of Correlation of {} Usage".format(stats_col))
    plt.show()

In [ ]:
def print_all_cdfs_for_stats(data_df, col_name, col_title, stats_col, is_max=True):
    """Prints all of the CDFs for the correlations for `col_name` across all stats.
    
    Parameters
    ----------
    data_df: pd.DataFrame
        The pandas DataFrame containing the distributions for the CDFs
    col_name: str
        A string representing the column for which the CDFs are generated.
    col_title: str
        A string representing the column title for the CDFs.
    stats_col: str
        A string representing the name of the column for which all statistics
        are generated.
    is_max: bool
        A boolean indicating whether `col_name` corresponds to a maximum column.
    
    Returns
    -------
    None
    
    """
    if is_max:
        col_names = ["{0}_{1}".format(col_name, stat) for stat in specs.MAX_STATS_COLS]
    else:
        col_names = ["{0}_{1}".format(col_name, stat) for stat in specs.AVG_STATS_COLS]
    for name in col_names:
        print(name)
        plot_lagged_cdfs(data_df, name, col_title, stats_col)
        print("------------------------------------------------------------------------")

In [ ]:
def plot_all_lagged_correlation_cdfs(stat_df, stats_col):
    """Plots all of the CDFs for the correlations in `stat_df`.
    
    Parameters
    ----------
    stat_df: pd.DataFrame
        The pandas DataFrame containing the distributions for the CDFs.
    stats_col: str
        A string representing the name of the column for which all statistics
        are generated.
    
    Returns
    -------
    None
    
    """
    for idx in range(len(specs.RAW_TIME_SERIES_COLS)):
        col_name = specs.RAW_TIME_SERIES_NAMES[idx]
        is_max = (col_name.split()[0] == "Maximum")
        print_all_cdfs_for_stats(stat_df, specs.RAW_TIME_SERIES_COLS[idx], col_name, stats_col, is_max)

In [ ]:
def plot_causal_cdfs(data_df, col_name, test_name, col_title):
    """Plots the causal CDFs for `col_name`.
    
    Parameters
    ----------
    data_df: pd.DataFrame
        The pandas DataFrame containing the distributions for the CDFs
    col_name: str
        A string representing the column for which the CDFs are generated.
    test_name: str
        A string representing the name of the causality test.
    col_title: str
        A string representing the column title for the CDFs.
    
    Returns
    -------
    None
    
    """
    fig, axes = plt.subplots(3, figsize=(20, 20))
    lags = [1, 2, 3]
    col_names = ["causal_{0}_{1}_{2}".format(col_name, test_name, lag) for lag in lags]
    colors = ["blue", "black", "green"]
    for idx in range(3):
        data_vals = data_df[col_names[idx]].abs().values
        data_vals = data_vals[~np.isnan(data_vals)]
        plotting.plot_cumulative_distribution_function(data_df[col_names[idx]].abs().values, axes[idx],
                                                       "{0} Lag {1}".format(col_title, lags[idx]), colors[idx],
                                                       "CDF of Causation P-Values")
    plt.show()

In [ ]:
def plot_all_causal_cdfs(stat_df, causal_cols, causal_titles, test_name):
    """Plots all causal CDFs for the columns in `causal_cols`.
    
    Parameters
    ----------
    stat_df: pd.DataFrame
        The pandas DataFrame containing the distributions for the CDFs
    causal_cols: list
        A list of strings denoting the names of the columns used in the causality tests.
    causal_titles: list
        A list of strings representing the titles for the CDFs.
    test_name: str
        A string representing the name of the causality test.
    
    Returns
    -------
    None
    
    """
    for idx in range(len(causal_cols)):
        print(causal_cols[idx])
        plot_causal_cdfs(stat_df, "{}_ts".format(causal_cols[idx]), test_name, causal_titles[idx])

In [ ]:
def summarize_statistical_results(input_dir, target_col, causal_test, stats_col):
    """Summarizes the statistical results in `input_dir` for `target_col`.
    
    Parameters
    ----------
    input_dir: str
        A string representing the path of the directory containing the results.
    target_col: str
        A string representing the name of the column serving as the target variable for analysis.
    causal_test: str
        A string representing the name of the causality test being summarized.
    stats_col: str
        A string representing the name of the column for which all statistics
        are generated.
    
    Returns
    -------
    None
    
    """
    stats_data = pd.read_csv(os.path.join(input_dir, "trace_stats.csv"))
    target_idx = specs.RAW_TIME_SERIES_COLS.index(target_col)
    causal_cols = [specs.RAW_TIME_SERIES_COLS[idx] 
                   for idx in range(len(specs.RAW_TIME_SERIES_COLS)) if idx != target_idx]
    causal_titles = [specs.RAW_TIME_SERIES_NAMES[idx]
                     for idx in range(len(specs.RAW_TIME_SERIES_NAMES)) if idx != target_idx]
    print_statistical_analysis_summary(stats_data, target_col, causal_cols)
    print()
    plot_utilization_cdf(stats_data, stats_col.split()[-1])
    plot_all_correlations_across_all_lags(stats_data, stats_col)
    plot_all_lagged_correlation_cdfs(stats_data, stats_col)
    print()
    plot_all_causal_cdfs(stats_data, causal_cols, causal_titles, causal_test)
    return stats_data

In [ ]:
stats_dfs = {}

### Maximum Memory Usage - 3 Period Aggregation

In [ ]:
max_mem_3_dir = "/Users/mattb/Desktop/Courses/MemoryAutoScaling/output_data/max_mem_3"
stats_dfs['max_mem_agg_3'] = summarize_statistical_results(max_mem_3_dir, specs.MAX_MEM_COL, "ssr_chi2test", "Maximum Memory")

### Maximum CPU Usage - 3 Period Aggregation

In [ ]:
max_cpu_3_dir = "/Users/mattb/Desktop/Courses/MemoryAutoScaling/output_data/max_cpu_3"
stats_dfs['max_cpu_agg_3'] = summarize_statistical_results(max_cpu_3_dir, specs.MAX_CPU_COL, "ssr_chi2test", "Maximum CPU")

### Maximum Memory Usage - 24 Period Aggregation

In [ ]:
max_mem_24_dir = "/Users/mattb/Desktop/Courses/MemoryAutoScaling/output_data/max_mem_24"
stats_dfs['max_mem_agg_24'] = summarize_statistical_results(max_mem_24_dir, specs.MAX_MEM_COL, "ssr_chi2test", "Maximum Memory")

### Maximum CPU Usage - 24 Period Aggregation

In [ ]:
max_cpu_24_dir = "/Users/mattb/Desktop/Courses/MemoryAutoScaling/output_data/max_cpu_24"
stats_dfs['max_cpu_agg_24'] = summarize_statistical_results(max_cpu_24_dir, specs.MAX_CPU_COL, "ssr_chi2test", "Maximum CPU")

In [ ]:
analysis.get_stationary_results_df(stats_dfs)